In [ ]:
# !pip install openai

In [15]:
# importing relevant libraries
import numpy as np
import pandas as pd
import openai
import json
import configparser
from google.cloud import bigquery

# Set up the BigQuery client
client = bigquery.Client()
### GPT Credentials
config = configparser.ConfigParser()
config.read('config.ini')

# deployemnet_name = config['gpt_config']['deployemnet_name']
# openai.api_type = config['gpt_config']['api_type']
# openai.api_key = config['gpt_config']['api_key'] 
# openai.api_base = config['gpt_config']['api_base'] 
# openai.api_version = config['gpt_config']['api_version'] 

deployemnet_name="gpt-4-32k"
openai.api_type="azure"
openai.api_key="dcf1dbc17e2f4d999be7ec5a35578915" #os.genenv("OPENAI_API_KEY")
openai.api_base="https://zee-chatgpt4-prod.openai.azure.com/" #endpoint
openai.api_version="2023-03-15-preview"

# deployemnet_name="reco-studio-gpt4"
# openai.api_type="azure"
# openai.api_key="6f9ddb7e06b74ef9858166668e4e7faa" #os.genenv("OPENAI_API_KEY")
# openai.api_base="https://aai-zee5-sb-ci1-recostudio-0001.openai.azure.com/" #endpoint
# openai.api_version="2023-03-15-preview"

ModuleNotFoundError: No module named 'openai'

In [17]:
def data_preparation(cms, data):
    
    """ Prepare the Data for Gen AI Rails Generation """
    
    ## CMS Asset Info
    id_name_mapping = cms.set_index('asset_id')['asset_name'].to_dict()
    id_genre_mapping = cms.set_index('asset_id')['genre'].to_dict()
    id_lang_mapping = cms.set_index('asset_id')['AudioLanguages'].to_dict()
    name_id_mapping = cms.set_index('asset_name')['asset_id'].to_dict()
    id_genai = cms.set_index('asset_id')[['asset_name', 'genre', 'AudioLanguages']].to_dict()

    ## Get Complete Data
    data['asset_name']= data.assetId.apply(lambda x: id_name_mapping[x] if x in id_name_mapping.keys() else None)
    data.dropna(inplace=True)

    ## Get Embeddings and Recos for every Content
    id_asset_mapping = data_mapping.set_index('id')['assetId'].to_dict()
    data['recommendation_asset_ids'] = data.recommendation.apply(lambda x: [id_asset_mapping[i] for i in x if i in id_asset_mapping.keys()])
    data.head()

    ## Reco Asset Names
    data['recommendation_names']= data.recommendation_asset_ids.apply(lambda x: [id_name_mapping[i] for i in x if i in id_name_mapping.keys()])

    ## Get Asset Id from CMS
    data['asset_ids_in_cms']= data.recommendation_asset_ids.apply(lambda x: [i for i in x if i in id_name_mapping.keys()])

    ## Add Features
    data['len_recommendation']= data.recommendation.apply(lambda x: len(x))
    data['len_recommendation_asset_ids']= data.recommendation_asset_ids.apply(lambda x: len(x))
    data['len_recommendation_names']= data.recommendation_names.apply(lambda x: len(x))
    data['len_asset_ids_in_cms']= data.asset_ids_in_cms.apply(lambda x: len(x))
    data.head()

    ## Remove Content without Genres
    cms= cms[~cms.genre.isnull()].reset_index(drop = True)
    cms.genre= cms.genre.apply(lambda x: x.replace('"', '').split(','))

    ## Create Asset - Genre Dict
    name_gen_mapping = cms.set_index('asset_name')['genre'].to_dict()
    cms['input_for_genai']= cms[["asset_name", "genre"]].apply(lambda x: tuple(x), axis = 1)
    assetid_genai_input_mapping= cms.set_index('asset_id')['input_for_genai'].to_dict()

    ## Asset Name Dict 
    movie_to_recommendations= data.set_index('assetId')['asset_ids_in_cms'].to_dict()
    
    return(movie_to_recommendations, assetid_genai_input_mapping, id_genre_mapping, id_name_mapping, name_id_mapping)


## Popularity Biased
def popularity_items_on_zee5():
    
    # Construct a parameterized SQL query that uses the variable
    
    query_1 = """
        select asset_id,asset_name, duration_sec,count(distinct viewerid) as n_users 
        from 
        (select a.viewerid, b.asset_id, b.asset_name, b.duration_sec
        from zee-prj-udp.udp_silver.conviva_partitioned a 
        join zee-prj-udp.udp_silver.cms b 
        on b.asset_id = a.sessiontag_contentid  
        where lower(subtype) = 'movie' 
        and cast(videoview_dateid_ist as date) >= DATE_ADD(CURRENT_DATE(),INTERVAL -90 day))
        group by 1,2,3
        order by count(distinct viewerid) desc
    """

    
    # Set the query parameters
    job_config = bigquery.QueryJobConfig()
    # Execute the query and retrieve the results as a Pandas DataFrame
    popular_movies = client.query(query_1, job_config=job_config).to_dataframe()
    return popular_movies
    

In [18]:
# def generate_movie_recommendations(watched_movie, subject_matter, genres, list_movies):
#     messages= [
#             {  
#                "role": "system",
#                 "content": f"""an user has watched movie {watched_movie} whose subject matter is {subject_matter} and genre is {genres}. Given a list of similar movies with their title, subject matter and genre, your task is to generate 3 collections at least 40 distinct movies which are similar to watched movie, ensuring that movies must not repeat within and across the collections. 
#                 Each collection should have similarity in the movies it contains and on that basis, give prolific name for each collection. Input Format Example - [('Movie Title', 'subject matter', 'genres')].
#                 {list_movies}
#                 The output should be in json format with keys as "collections" and "movies". "Collections" should contain list of collection names and "movies" should contain list of movie titles corresponding collection name as values. 
#                 Please make sure that output is a valid json.
# """
#             }
#         ]

#     response = openai.ChatCompletion.create(messages=messages,temperature=1.0,max_tokens=20000, top_p=1, frequency_penalty=0,
#   presence_penalty=0,engine=deployemnet_name)
    
#     #print(response)
#     #response = openai.ChatCompletion.create(prompt=prompt)
#     #Remove non-ASCII characters from the response
    
#     assistant_messages = response['choices'][0]['message']['content']
#     movie_recommendations_start = assistant_messages.find("{")
#     movie_recommendations_json = assistant_messages[movie_recommendations_start:]
    
#     return assistant_messages



In [19]:

def get_recommendations(movie_id, popular_df, movie_name, id_name_mapping, movie_to_recommendations, metadata_sm, metadata_g, n, name_id_mapping):
    
    """ Hit GPT API and Get final Recos"""
    
    list_movies = [(id_name_mapping[i],metadata_sm[i], metadata_g[i]) for i in movie_to_recommendations[movie_id] if (i in metadata_sm.keys()) & (i in metadata_g.keys())]
    result = eval(generate_movie_recommendations(watched_movie=movie_name, 
                                                  subject_matter = metadata_sm[movie_id],
                                                  genres = metadata_g[movie_id],
                                                  list_movies=list_movies[:200]))
    print(list_movies)
    ### Prepare Recos for given movie
    df = pd.DataFrame(result['movies'].items())
    df['watched_movie'] = [movie_name]*n
    df.rename(columns={0: 'collection_name', 1: 'movies'}, inplace=True)

    ## Replace "'s"
    df["movies"] = df.movies.apply(lambda x: [i.replace("'", "") for i in x])
    df = df.reset_index(drop = True)

    ## Add Movie Ids
    df['asset_ids']= df.movies.apply(lambda x: list(set([name_id_mapping[i] for i in x if i in name_id_mapping.keys()])))

    ## Get Popularity Data
    df['final_recos']= df.asset_ids.apply(lambda x: popular_df[popular_df.asset_id.isin(x)].asset_name.values.tolist())

    ## Get Ids
    df['final_recos_ids']= df.final_recos.apply(lambda x: [name_id_mapping[i] for i in x])
    
    return(list_movies, df)

In [20]:
### Read Contexual Data
data= pd.read_csv('gs://ai-ml-team/Prod_CRE/Output/CRE/Contextual/V1/2024-01-11/data.csv', sep='^')
data.recommendation = data.recommendation.apply(lambda x: [int(i.split(':')[0]) for i in x.split('~')])

In [21]:
%%bigquery data_mapping
SELECT id, assetId FROM `zee-prj-udp.zee_reco_aiml.contextual_w2v_embeddings`


Query is running:   0%|          |

Downloading:   0%|          |

In [22]:
%%bigquery cms 
select tvshowid, asset_id, asset_name, web_description, app_description,genre, AudioLanguages 
from `zee-prj-udp.udp_silver.cms` where lower(subtype) = 'movie' 


Query is running:   0%|          |

Downloading:   0%|          |

In [23]:
%%bigquery movies_metadata
select * from `prj-ds-dev-2a6a.Zee_aiml_reco_consumer.item_genai_movies_metadata`


Query is running:   0%|          |

Downloading:   0%|          |

In [24]:
## Metadata Dictionaries
id_name_mapping = movies_metadata.set_index('tvshowid')['asset_name'].to_dict()
id_genre_mapping = movies_metadata.set_index('tvshowid')['genres'].to_dict()
id_emotion_mapping = movies_metadata.set_index('tvshowid')['emotions'].to_dict()
id_keywords_mapping = movies_metadata.set_index('tvshowid')['keywords'].to_dict()
id_theme_mapping = movies_metadata.set_index('tvshowid')['themes'].to_dict()
id_subject_matter_mapping = movies_metadata.set_index('tvshowid')['subject_matter'].to_dict()
id_real_life_elements_mapping = movies_metadata.set_index('tvshowid')['real_life_elements'].to_dict()
id_audience_segment_mapping = movies_metadata.set_index('tvshowid')['audience_segments'].to_dict()
name_id_mapping = movies_metadata.set_index('asset_name')['tvshowid'].to_dict()


In [10]:
def generate_movie_recommendations(watched_movie, subject_matter, genres, list_movies):
    messages= [
            {  
               "role": "system",
                "content": f"""As a movie recommender system, your task is to generate 3 collections of movies for an user who has watched movie {watched_movie} whose subject is {subject_matter} and genre is {genres}.
                Points to remember while creating collections: 
                - Each collection must have at least 40 distinct movies which are similar to watched movie. 
                - Make sure that movies must not repeat across collections.
                - Make sure that movies must not repeat in the same collections.
                - Each collection should have similarity in the movies it contains and on that basis, give prolific name for each collection.
                Here is the list of similar movies with their title, subject matter and genre out of which you have to create 3 collections of movies.
                Input Format Example - [('Movie Title', 'subject matter', 'genres')].
                {list_movies}
                The output should be in json format with keys as "collections" and "movies". "Collections" should contain list of collection names and "movies" should contain list of movie titles corresponding collection name as values. 
                Please make sure that output is a valid json.
"""
            }
        ]

    response = openai.ChatCompletion.create(messages=messages,temperature=1.0,max_tokens=20000, top_p=1, frequency_penalty=0,
  presence_penalty=0,engine=deployemnet_name)
    
    #print(response)
    #response = openai.ChatCompletion.create(prompt=prompt)
    #Remove non-ASCII characters from the response
    
    assistant_messages = response['choices'][0]['message']['content']
    movie_recommendations_start = assistant_messages.find("{")
    movie_recommendations_json = assistant_messages[movie_recommendations_start:]
    
    return assistant_messages



In [11]:
movie_id = '0-0-movie_1511761297'
movie_name = id_name_mapping[movie_id]
subject_matter = id_subject_matter_mapping[movie_id]

## Get Input Data
movie_to_recommendations, assetid_genai_input_mapping, id_genre_mapping, id_name_mapping, name_id_mapping = data_preparation(cms, data)

## Get Popularity Data
popular_df = popularity_items_on_zee5()

genre = id_genre_mapping[movie_id]
n = 3
list_movies, df = get_recommendations(movie_id, popular_df, movie_name, id_name_mapping, movie_to_recommendations, id_subject_matter_mapping, id_genre_mapping, n, name_id_mapping)
# print(movie_name, subject_matter, genre)
df[["collection_name", "watched_movie", "recos"]]
# .to_csv("GenAI_Recos_Consumption_Page_metadata_gadar.csv", index = False)

# df["movies_len"] = df["movies"].apply(lambda x: len(x))
# df["asset_ids_len"] = df["asset_ids"].apply(lambda x: len(x))
# df["final_recos_len"] = df["final_recos"].apply(lambda x: len(x))
# df["final_recos_ids_len"] = df["final_recos_ids"].apply(lambda x: len(x))

df

[('Shaadi Mein Zaroor Aana', "['Marriage', 'Education', 'Career aspirations', 'Dowry']", 'Comedy,Romance'), ('14 Phere', "['Love', 'Inter-Caste Marriage', 'Family Conflict', 'Deception']", 'Comedy,Romance'), ('Lovey Dovey', "['family drama', 'house renovation', 'economic crisis']", 'Comedy,Romance'), ('Yamla Pagla Deewana Phir se', "['Ayurveda', 'Pharmaceutical commercialization', 'Family dynamics', 'Legal battles']", 'Action,Comedy'), ('Driving Me Crazy', "['Inter-caste marriage', 'Elopement', 'Terrorism', 'Jungle survival']", 'Comedy,Romance'), ('BHK : Bhalla @ Halla.Kom', "['Marriage', 'House hunting', 'Father-daughter relationship']", 'Twist and Turns,Comedy'), ('Ishq Forever', "['Political Appointment', 'Security Measures', 'Teenage Rebellion']", 'Twist and Turns,Romance'), ('Dagabaaz Piritiya', "['crime', 'investigation', 'pursuit', 'serial killing']", 'Romance'), ('Dil Juunglee', "['love story', 'pursuit of career', 'teacher-student relationship', 'Bollywood industry']", 'Rom Co

,collection_name,movies,watched_movie,asset_ids,final_recos,final_recos_ids
0,Aspirations Amidst Love and Tradition,"[14 Phere, Ishq Forever, London Babulu, Mitron...",Shaadi Mein Zaroor Aana,"[0-0-105974, 0-0-7035, 0-0-7655, 0-0-2082, 0-0...","[14 Phere, Tadka, Partner, Toilet: Ek Prem Kat...","[0-0-1z51198, 0-0-1z5246147, 0-0-partner, 0-0-..."
1,Struggles in Love and Career,"[Dono, Fatafat - A ZEE5 Original Movie, Chal M...",Shaadi Mein Zaroor Aana,"[0-0-1z5418888, 0-0-movie_1389847534, 0-0-4781...","[Dono, Biye Bibhrat, Rockstar, Cabaret, Maduve...","[0-0-1z5485742, 0-0-1z5418888, 0-0-105985, 0-0..."
2,Humorous Twists in Love and Family,"[Mitha Mitha, Waah Zindagi, Dagabaaz Piritiya,...",Shaadi Mein Zaroor Aana,"[0-0-7617, 0-0-7035, 0-0-movie_1148624952, 0-0...","[PellisandaD, Cabaret, Ajay Gadu, Maduveya Mam...","[0-0-1z5168309, 0-0-18540, 0-0-1z5496653, 0-0-..."


In [13]:
df["asset_id"] = '0-0-movie_1511761297'
df = df[["asset_id", "collection_name", "final_recos_ids"]]
df.columns = ["asset_id", "rail_title", "recos"]
df.to_csv("movies_consumption_page.csv", index = False)

In [31]:
df

NameError: name 'df' is not defined

In [25]:
for i in range(1, df.shape[0]+1):
    df[i-1:i].to_csv("Rail_"+str(i)+".csv", index = False)
    
# for row in df_bed.reset_index().values:
#     #print row
#     row[1:].tofile(str(row[0])+'.bed', sep="\t", format="%s")

In [ ]:
# ## Get Rail Level Popularity
# id_popularity_mapping = popular_df.set_index('asset_id')['n_users'].to_dict()
# df["Popularity"] = df.apply(lambda x: sum([id_popularity_mapping[i] for i in x["final_recos_ids"] if i in id_popularity_mapping.keys()]), axis = 1)
# df = df.sort_values("Popularity", ascending=False).reset_index(drop = True)
# df["Rank"] = range(1, df.shape[0]+1)
# df

In [ ]:
# ## Within Rail Duplicate removal
# df["dedups_recos"] = None
# for i in range(df.shape[0]):
#     ids_within = []
#     for item in df["final_recos_ids"][i]:
#         if item not in ids_within:
#             ids_within.append(item)
#     df["dedups_recos"][i] = ids_within
        


In [ ]:
# ## Remove Duplicates across rail
# df["dedups_recos_final"] = None
# ls = []
# for i in range(df.shape[0]):
#     ids_within = []
#     for item in df["final_recos_ids"][i]:
#         if item not in ls:
#             ids_within.append(item)
#             ls.append(item)
#     df["dedups_recos_final"][i] = ids_within
            
    

In [45]:
from datetime import date
#reading rail files
Rail1= pd.read_csv('Rail_1-2.csv')
Rail2= pd.read_csv('Rail_2-2.csv')
Rail3= pd.read_csv('Rail_3-2.csv')

d = date.today().strftime('%Y-%m-%d')
print('gs://ai-ml-team/Prod_CRE/Output/CRE/Consumption_Page/Movies/Rail1/V1/{d}/Rail1.parquet')

Rail1["recos"] = [",".join(i for i in eval(Rail1["recos"][0]))]
# Rail2["recos"] = ",".join(i for i in eval(Rail2["recos"][0]))
# Rail3["recos"] = ",".join(i for i in eval(Rail3["recos"][0]))

# # #uploading rail files
# Rail1.to_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/Consumption_Page/Movies/Rail1/V1/{d}/Rail1.parquet', index= False)
# Rail2.to_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/Consumption_Page/Movies/Rail2/V1/{d}/Rail2.parquet', index= False)
# Rail3.to_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/Consumption_Page/Movies/Rail3/V1/{d}/Rail3.parquet', index= False)

In [67]:
Rail1= pd.read_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/Consumption_Page/Movies/Rail1/V1/{d}/Rail1.parquet')
Rail2= pd.read_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/Consumption_Page/Movies/Rail2/V1/{d}/Rail2.parquet')
Rail3= pd.read_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/Consumption_Page/Movies/Rail3/V1/{d}/Rail3.parquet')

In [62]:
ls = []
for i in Rail1["recos"]:
    ls.append(",".join(j for j in eval(i)))
Rail1["recos"] = ls
Rail1    

In [64]:
ls = []
for i in Rail2["recos"]:
    ls.append(",".join(j for j in eval(i)))
Rail2["recos"] = ls
Rail2

,asset_id,rail_title,recos
0,0-0-1z5485742,Light-Hearted Crime Comedies,"0-0-1z5334341,0-0-1z544916,0-0-1z5323051,0-0-1..."
1,0-0-1z5168309,"Love, Laughter and Life","0-0-1z5351501,0-0-1z5485742,0-0-movie_15117612..."
2,0-0-1z5418888,"Culture, Family and Society Collection","0-0-1z5318993,0-0-1z5246147,0-0-1z5430995,0-0-..."


In [65]:
ls = []
for i in Rail3["recos"]:
    ls.append(",".join(j for j in eval(i)))
Rail3["recos"] = ls
Rail3 

,asset_id,rail_title,recos
0,0-0-1z5485742,Mature Relationship Dramas,"0-0-1z5351501,0-0-1z5485754,0-0-movie_15117612..."
1,0-0-1z5168309,Comedy in Conflict,"0-0-1z5387314,0-0-1z5465753,0-0-1z5339383,0-0-..."
2,0-0-1z5418888,Thrilling and Adventurous Collection,"0-0-1z5228515,0-0-1z5277662,0-0-1z5458253,0-0-..."


In [66]:
# #uploading rail files
Rail1.to_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/Consumption_Page/Movies/Rail1/V1/{d}/Rail1.parquet', index= False)
Rail2.to_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/Consumption_Page/Movies/Rail2/V1/{d}/Rail2.parquet', index= False)
Rail3.to_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/Consumption_Page/Movies/Rail3/V1/{d}/Rail3.parquet', index= False)

In [25]:
Rail1

,Cohort,Rail_Name,tvshowid
0,C18,Biographical Tales,"0-0-1z5453981,0-0-1z5161064,0-0-16649,0-0-1z54..."
1,C22,Rural Rhapsody,"0-0-1z5418670,0-0-1z5368704,0-0-1z5387314,0-0-..."
2,C34,Family Feuds and Romantic Entanglements,"0-0-1z5418670,0-0-1z5368704,0-0-1z5387314,0-0-..."
3,C49,Justice and Legal Struggles,"0-0-1z5359029,0-0-1z5419208,0-0-1z5391348,0-0-..."
4,C16,Fight for Love,"0-0-1z5368704,0-0-1z5335005,0-0-1z5168593,0-0-..."
5,C30,Historical and Political Drama,"0-0-gadarekpremkatha,0-0-1z5133458,0-0-39320,0..."
6,C12,Twisted Justice,"0-0-1z5359029,0-0-1z5419208,0-0-1z5391348,0-0-..."
7,C41,Drama of Relationships & Love,"0-0-1z5419208,0-0-1z5391534,0-0-1z5418888,0-0-..."
8,C29,Family Fibers,"0-0-1z5418670,0-0-1z5368704,0-0-1z5419278,0-0-..."
9,C2,Inspirational Family Rotations,"0-0-1z5418670,0-0-1z5368704,0-0-1z5387314,0-0-..."


In [34]:
import pandas as pd
Rail1= pd.read_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/For_You_Page_GenAI_Rails/Rail1/V1/2024-01-23/Rail1.parquet')
Rail2= pd.read_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/For_You_Page_GenAI_Rails/Rail2/V1/2024-01-23/Rail2.parquet')
Rail3= pd.read_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/For_You_Page_GenAI_Rails/Rail3/V1/2024-01-23/Rail3.parquet')
Rail4= pd.read_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/For_You_Page_GenAI_Rails/Rail4/V1/2024-01-23/Rail4.parquet')

In [13]:
# import os
# os.getcwd()

'/home/jupyter/Consumption Page - Gen AI'

In [14]:
Rail1.to_csv('/home/jupyter/Rails-Generation-GenAI/For You Page /Rail1.csv', index = False)
Rail2.to_csv('/home/jupyter/Rails-Generation-GenAI/For You Page /Rail2.csv', index = False)
Rail3.to_csv('/home/jupyter/Rails-Generation-GenAI/For You Page /Rail3.csv', index = False)
Rail4.to_csv('/home/jupyter/Rails-Generation-GenAI/For You Page /Rail4.csv', index = False)

In [29]:
id_name_mapping["0-0-1z5334341"]

'Mrs Undercover'

In [31]:
import pandas as pd
Rail1= pd.read_csv(f'/home/jupyter/Rails-Generation-GenAI/For You Page /Rail1.csv')
Rail2= pd.read_csv(f'/home/jupyter/Rails-Generation-GenAI/For You Page /Rail2.csv')
Rail3= pd.read_csv(f'/home/jupyter/Rails-Generation-GenAI/For You Page /Rail3.csv')
Rail4= pd.read_csv(f'/home/jupyter/Rails-Generation-GenAI/For You Page /Rail4.csv')


In [33]:
Rail1.to_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/For_You_Page_GenAI_Rails/Rail1/V1/2024-01-23/Rail1.parquet')
Rail2.to_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/For_You_Page_GenAI_Rails/Rail2/V1/2024-01-23/Rail2.parquet')
Rail3.to_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/For_You_Page_GenAI_Rails/Rail3/V1/2024-01-23/Rail3.parquet')
Rail4.to_parquet(f'gs://ai-ml-team/Prod_CRE/Output/CRE/For_You_Page_GenAI_Rails/Rail4/V1/2024-01-23/Rail4.parquet')